In [1]:
#%% Modules
import pyvisa as visa
import awg_support as awg
import numpy as np 
import matplotlib.pyplot as plt
import csv
import time
import serial

In [2]:
#%% Initialisation
# Setup for the tools
ID_awg = 'USB0::0xF4EC::0x1102::SDG2XFBC7R0482::INSTR'
rm = awg.resourcer()
inst_awg = awg.initialise(ID_awg, rm)

In [3]:
#noise parameters
sd = 1.1
Ts = 1/8e6
tw = np.arange(0, 1, Ts)

In [4]:
awg.set_output_state(inst_awg,channel=2,val="ON")
wave = np.random.normal(0,sd,size=np.shape(tw))
awg.arb_wave_constructor(inst_awg, wave, tw,freq=80e3,channel=2)

In [5]:
def run(ser):
    bit = []
    count = 0
    while True:
        if ser.in_waiting > 0:
                try:
                    line = ser.readline()
                    val = line.decode('utf-8').strip()  # Attempt to decode as UTF-8
                    #print(val)
                    if(val == "0" or val == "1"):
                        bit.append(int(val))
                        count += 1
                        if(count == 1e4):
                            return bit
                except ValueError:
                    continue
                except UnicodeDecodeError:
                    print("error")
    return bit

In [6]:
ser = serial.Serial('COM3', 115200, timeout=1)
#collecting 1.5lk points
#collecting 10000 points per cycle
#similarly we collect 100 such points
TRN = []
for i in range(15):
    TRN_bits = run(ser)
    TRN.append(TRN_bits)
    print("1e4 bits done: bit probability = ", np.mean(TRN_bits))

1e4 bits done: bit probability =  0.5277
1e4 bits done: bit probability =  0.5273
1e4 bits done: bit probability =  0.5282
1e4 bits done: bit probability =  0.527
1e4 bits done: bit probability =  0.5229
1e4 bits done: bit probability =  0.522
1e4 bits done: bit probability =  0.5334
1e4 bits done: bit probability =  0.5242
1e4 bits done: bit probability =  0.5246
1e4 bits done: bit probability =  0.5168
1e4 bits done: bit probability =  0.5146
1e4 bits done: bit probability =  0.5193
1e4 bits done: bit probability =  0.5151
1e4 bits done: bit probability =  0.5248
1e4 bits done: bit probability =  0.5177


In [7]:
ser.close()
awg.set_output_state(inst_awg,channel=2,val="OFF")
awg.set_output_state(inst_awg,channel=1,val="OFF")
awg.deinitialise(inst_awg)
awg.deinitialise(rm)

In [38]:
# bitstream = ''.join(f"{int(h, 16):016b}" for h in TRNG)  # Each hex is converted to 16-bit binary
# bit_array = [int(bit) for bit in bitstream]
# plt.figure(figsize=(20,5))
# plt.plot(bit_array)
# print("p = ", np.mean(bit_array))
# print("total no: TRNG = ", len(bit_array))

In [ ]:
# np.savez("TRNG_openloop_p_0.5_sd_1.1_vbias_0_137.npz", TRN)

In [40]:
# def hex_file_to_bitstream(file_path):
#     with open(file_path, "r") as file:
#         hex_values = [line.strip() for line in file if line.strip()]  # Read and clean lines

#     # Convert each hex value to a 16-bit binary representation
#     bitstream = ''.join(f"{int(h, 16):016b}" for h in hex_values)

#     # Convert binary string into a list of bits (0s and 1s)
#     bit_array = [int(bit) for bit in bitstream]

#     return bit_array

# # Example usage
# file_path = "openloop_bits_2_4_25.txt"  # Replace with your actual file path
# bit_array = hex_file_to_bitstream(file_path)
# plt.figure(figsize=(20,5))
# plt.plot(bit_array[0:1000])
# print("p = ", np.mean(bit_array))
# print("total no: TRNG = ", len(bit_array))

# output_file = "bitstream_openloop_2_4_25.txt"
# with open(output_file, "w") as file:
#         file.write(''.join(map(str, bit_array)) + "\n")  # Save as continuous bitstream


In [2]:
TRN = np.load("TRNG_openloop_p_0.5_sd_1.1_vbias_0_137.npz")
TRN = TRN['arr_0']

In [3]:
TRN_flat = np.array(TRN).flatten()

In [4]:
bit_string = ''.join(TRN_flat.astype(str))
# Save to a text file
with open('TRNG_openloop_p_0.5_sd_1.1_vbias_0_137.txt', 'w') as f:
    f.write(bit_string)